# Unificación de datos y creación de los datasets

En este cuaderno, por fin, llevamos a cabo la uniificación de los datos electorales y sociodemográficos en un solo dataset por cada elección. 

Tomaremos como base el df con los datos electorales e iremos añadiendo los del afiliación a la SS, paro y edades y demás. Aprovecharemos para definir columnas que pueden resultar interesantes para llevar a cabo diversas modelizaciones.

Una de las virtudes de los datasets es que no hay una variable objetivo obvia, sino que se pueden elegir en teoría infinitas.

Comenzamos cargando las librerías y el dataset de los resultados electorales, en este caso el de las elecciones de noviembre de 2011.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import boto3

BUCKET_NAME = 'electomedia' 

# sustituir por credenciales de acceso. 
s3 = boto3.resource('s3', aws_access_key_id = 'XXXXXX', 
                          aws_secret_access_key= 'XXXXXXXX')

In [3]:
import botocore.exceptions

KEY = 'INE/resultados_comp_N11_territ.txt' 

try:
  
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'resultados_comp_N11_territ.txt')

except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [4]:
strings = {'Sección' : 'str', 'cod_ccaa' : 'str', 'cod_prov' : 'str', 'cod_mun' : 'str'}

In [5]:
votos = pd.read_csv('resultados_comp_N11_territ.txt', dtype = strings)

In [6]:
votos

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,0,0,0,0,0,0,0,0,0,10
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,0,0,0,0,0,0,0,0,0,9
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,0,0,0,0,0,0,0,0,0,6
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,0,0,0,0,0,0,0,0,0,18
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,0,0,0,0,0,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35952,022011111195200108010,19,52,52001,Melilla,Melilla,Melilla,1208,775,8,...,0,0,0,0,0,0,0,0,3,19
35953,022011111195200108011,19,52,52001,Melilla,Melilla,Melilla,1374,811,8,...,0,0,0,0,0,0,0,0,3,19
35954,022011111195200108012,19,52,52001,Melilla,Melilla,Melilla,1874,1183,13,...,0,0,0,0,0,0,0,0,5,30
35955,022011111195200108013,19,52,52001,Melilla,Melilla,Melilla,1167,637,7,...,0,0,0,0,0,0,0,0,2,12


In [7]:
votos['Amaiur'].sum()

333762

In [8]:
votos.columns

Index(['Sección', 'cod_ccaa', 'cod_prov', 'cod_mun', 'CCAA', 'Provincia',
       'Municipio', 'Censo_Esc', 'Votos_Total', 'Nulos', 'Votos_Válidos',
       'Blanco', 'V_Cand', 'PP', 'PSOE', 'Cs', 'UP', 'IU', 'VOX', 'UPyD', 'MP',
       'CiU', 'ERC', 'JxC', 'CUP', 'DiL', 'PNV', 'Bildu', 'Amaiur', 'CC', 'FA',
       'TE', 'BNG', 'PRC', 'GBai', 'Compromis', 'PACMA', 'Otros'],
      dtype='object')

In [9]:
strings = {'Sección' : 'str', 'cod_ccaa' : 'str', 'cod_prov' : 'str', 'cod_mun' : 'str'}

Lo primero que hacemos es definir una columna con el código de la sección, 'cod_sec', que son simplemente los 10 últimos dígitos de la columna Sección. Eso nos permitirá hacer los merge de forma más cómoda.

In [10]:
votos['cod_sec'] = votos['Sección'].str[-10:]

In [11]:
votos

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros,cod_sec
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,0,0,0,0,0,0,0,0,10,0400101001
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,0,0,0,0,0,0,0,0,9,0400201001
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,0,0,0,0,0,0,0,0,6,0400301001
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,0,0,0,0,0,0,0,0,18,0400301002
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,0,0,0,0,0,0,0,0,7,0400301003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35952,022011111195200108010,19,52,52001,Melilla,Melilla,Melilla,1208,775,8,...,0,0,0,0,0,0,0,3,19,5200108010
35953,022011111195200108011,19,52,52001,Melilla,Melilla,Melilla,1374,811,8,...,0,0,0,0,0,0,0,3,19,5200108011
35954,022011111195200108012,19,52,52001,Melilla,Melilla,Melilla,1874,1183,13,...,0,0,0,0,0,0,0,5,30,5200108012
35955,022011111195200108013,19,52,52001,Melilla,Melilla,Melilla,1167,637,7,...,0,0,0,0,0,0,0,2,12,5200108013


Ahora cargamos el dataset de edades, que vemos que tiene también la columna del código de secciones.

In [12]:
strings_edades = {'cod_sec' : 'str'}

In [13]:
import botocore.exceptions

KEY = 'INE/2012_df_edades_secciones.csv' 

try:
  
  s3.Bucket(BUCKET_NAME).download_file(KEY, '2012_df_edades_secciones.csv')

except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [14]:
edades = pd.read_csv('2012_df_edades_secciones.csv', dtype = strings_edades)

In [15]:
edades

,cod_sec,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,...,70-74,75-79,80-84,85-89,90-94,95-99,100 y más,Total,Hombres,Mujeres
0,0100101001,119,103,65,47,45,68,117,215,154,...,30,26,13,4,7,1,0,1341,698,643
1,0100101002,131,120,89,58,59,75,152,201,176,...,32,27,22,10,6,0,0,1528,786,742
2,0100201001,91,60,81,86,104,137,128,118,130,...,68,79,71,42,15,1,0,1979,1012,967
3,0100201002,64,44,52,75,97,110,114,98,105,...,52,70,49,31,9,5,0,1702,894,808
4,0100201003,46,43,41,45,45,80,78,76,69,...,41,31,43,23,12,0,0,1042,517,525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35973,5200108010,74,60,84,80,115,129,120,92,93,...,49,42,37,11,2,0,0,1480,735,745
35974,5200108011,148,155,148,169,162,144,147,140,167,...,33,36,19,13,4,0,0,2107,1057,1050
35975,5200108012,143,145,125,118,129,196,249,215,219,...,75,70,45,30,4,1,0,2532,1307,1225
35976,5200108013,108,84,80,81,150,193,189,165,124,...,33,34,21,2,5,1,0,1762,974,788


Pasamos a hacer el merge correspondiente de votos y edades a partir del código de la sección.

In [16]:
votos_edades = pd.merge(votos, edades, how="left", on=['cod_sec'])

In [17]:
votos_edades

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,70-74,75-79,80-84,85-89,90-94,95-99,100 y más,Total,Hombres,Mujeres
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,66.0,102.0,73.0,41.0,9.0,1.0,0.0,1465.0,765.0,700.0
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,80.0,85.0,58.0,35.0,8.0,0.0,1.0,1360.0,699.0,661.0
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,44.0,30.0,19.0,4.0,2.0,1.0,0.0,966.0,459.0,507.0
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,40.0,24.0,22.0,10.0,2.0,2.0,0.0,1579.0,789.0,790.0
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,40.0,58.0,28.0,12.0,6.0,0.0,0.0,1789.0,871.0,918.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35952,022011111195200108010,19,52,52001,Melilla,Melilla,Melilla,1208,775,8,...,49.0,42.0,37.0,11.0,2.0,0.0,0.0,1480.0,735.0,745.0
35953,022011111195200108011,19,52,52001,Melilla,Melilla,Melilla,1374,811,8,...,33.0,36.0,19.0,13.0,4.0,0.0,0.0,2107.0,1057.0,1050.0
35954,022011111195200108012,19,52,52001,Melilla,Melilla,Melilla,1874,1183,13,...,75.0,70.0,45.0,30.0,4.0,1.0,0.0,2532.0,1307.0,1225.0
35955,022011111195200108013,19,52,52001,Melilla,Melilla,Melilla,1167,637,7,...,33.0,34.0,21.0,2.0,5.0,1.0,0.0,1762.0,974.0,788.0


Ahora vamos con los datos de la afiiación a la Seguridad Social. Recordamos que teníamos los datos solo por municipios.

In [18]:
strings_seg_sol = {'cod_mun' : 'str'}

In [19]:
import botocore.exceptions

KEY = 'INE/seg_sol_muni_N11.txt' 

try:
  
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'seg_sol_muni_N11.txt')

except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [20]:
seg_sol = pd.read_csv('seg_sol_muni_N11.txt', dtype = strings_seg_sol)

In [21]:
seg_sol

,PROVINCIA,cod_mun,Municipio,Total_afiliados,prop_autonomos
0,01 ARABA/ÁLAVA,01001,ALEGRIA-DULANTZI,774,0.244186
1,01 ARABA/ÁLAVA,01002,AMURRIO,4476,0.139857
2,01 ARABA/ÁLAVA,01003,ARAMAIO,234,0.294872
3,01 ARABA/ÁLAVA,01004,ARTZINIEGA,399,0.255639
4,01 ARABA/ÁLAVA,01006,ARMIÑON,137,0.153285
...,...,...,...,...,...
8091,50 ZARAGOZA,50298,ZUERA,3589,0.264698
8092,50 ZARAGOZA,50901,BIEL-FUENCALDERAS,30,0.566667
8093,50 ZARAGOZA,50903,VILLAMAYOR DE GALLEGO,644,0.358696
8094,51 CEUTA,51001,CEUTA,19959,0.148605


In [22]:
seg_sol.columns

Index(['PROVINCIA', 'cod_mun', 'Municipio', 'Total_afiliados',
       'prop_autonomos'],
      dtype='object')

Para hacer el merge no nos hacen falta los datos de las provincias o municpios, solo el código de estos últimos para hacer el merge.

In [23]:
seg_sol_simp = seg_sol[['cod_mun', 'Total_afiliados',
       'prop_autonomos']]

In [24]:
seg_sol_simp

,cod_mun,Total_afiliados,prop_autonomos
0,01001,774,0.244186
1,01002,4476,0.139857
2,01003,234,0.294872
3,01004,399,0.255639
4,01006,137,0.153285
...,...,...,...
8091,50298,3589,0.264698
8092,50901,30,0.566667
8093,50903,644,0.358696
8094,51001,19959,0.148605


Hacemos el merge entre el dataset de votos y edades y este de la afiliación, mediante el código del municipio. 

Como se ve, vamos añadiendo datasets de forma progresiva.

In [25]:
votos_edades_segsol = pd.merge(votos_edades, seg_sol_simp, how="left", on=['cod_mun'])

In [26]:
votos_edades_segsol

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,80-84,85-89,90-94,95-99,100 y más,Total,Hombres,Mujeres,Total_afiliados,prop_autonomos
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,73.0,41.0,9.0,1.0,0.0,1465.0,765.0,700.0,281.0,0.241993
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,58.0,35.0,8.0,0.0,1.0,1360.0,699.0,661.0,387.0,0.224806
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,19.0,4.0,2.0,1.0,0.0,966.0,459.0,507.0,7161.0,0.357213
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,22.0,10.0,2.0,2.0,0.0,1579.0,789.0,790.0,7161.0,0.357213
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,28.0,12.0,6.0,0.0,0.0,1789.0,871.0,918.0,7161.0,0.357213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36058,022011111195200108010,19,52,52001,Melilla,Melilla,Melilla,1208,775,8,...,37.0,11.0,2.0,0.0,0.0,1480.0,735.0,745.0,19893.0,0.161816
36059,022011111195200108011,19,52,52001,Melilla,Melilla,Melilla,1374,811,8,...,19.0,13.0,4.0,0.0,0.0,2107.0,1057.0,1050.0,19893.0,0.161816
36060,022011111195200108012,19,52,52001,Melilla,Melilla,Melilla,1874,1183,13,...,45.0,30.0,4.0,1.0,0.0,2532.0,1307.0,1225.0,19893.0,0.161816
36061,022011111195200108013,19,52,52001,Melilla,Melilla,Melilla,1167,637,7,...,21.0,2.0,5.0,1.0,0.0,1762.0,974.0,788.0,19893.0,0.161816


Ahora vamos con los datos del paro, que también solo tienen datos del paro por municipios.

In [27]:
strings_paro = {'cod_mun' : 'str'}

In [28]:
import botocore.exceptions

KEY = 'INE/paro_muni_N11.txt' 

try:
  
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'paro_muni_N11.txt')

except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [29]:
paro = pd.read_csv('paro_muni_N11.txt', dtype = strings_seg_sol)

In [30]:
paro

,Código mes,cod_ccaa,CCAA,cod_prov,Provincia,cod_mun,Municipio,total Dtes Empleo,prop_hombres,prop_mayores_45
0,201111,1,Andalucía,4,Almería,04001,Abla,186,0.553763,0.500000
1,201111,1,Andalucía,4,Almería,04002,Abrucena,227,0.427313,0.594714
2,201111,1,Andalucía,4,Almería,04003,Adra,3821,0.525255,0.369013
3,201111,1,Andalucía,4,Almería,04004,Albánchez,62,0.516129,0.306452
4,201111,1,Andalucía,4,Almería,04005,Alboloduy,122,0.459016,0.524590
...,...,...,...,...,...,...,...,...,...,...
8112,201111,17,"Rioja, La",26,"Rioja, La",26180,Zarratón,19,0.526316,0.473684
8113,201111,17,"Rioja, La",26,"Rioja, La",26181,Zarzosa,0,0.500000,0.350000
8114,201111,17,"Rioja, La",26,"Rioja, La",26183,Zorraquín,5,0.800000,0.400000
8115,201111,18,Ceuta,51,Ceuta,51001,Ceuta,13947,0.420449,0.298344


In [31]:
paro.columns

Index(['Código mes ', 'cod_ccaa', 'CCAA', 'cod_prov', 'Provincia', 'cod_mun',
       'Municipio', 'total Dtes Empleo', 'prop_hombres', 'prop_mayores_45'],
      dtype='object')

Nos quedamos con los datos del paro necesarios y el código del municipio.

In [32]:
paro_simp = paro[['cod_mun', 'total Dtes Empleo', 'prop_hombres', 'prop_mayores_45']]

In [33]:
paro_simp

,cod_mun,total Dtes Empleo,prop_hombres,prop_mayores_45
0,04001,186,0.553763,0.500000
1,04002,227,0.427313,0.594714
2,04003,3821,0.525255,0.369013
3,04004,62,0.516129,0.306452
4,04005,122,0.459016,0.524590
...,...,...,...,...
8112,26180,19,0.526316,0.473684
8113,26181,0,0.500000,0.350000
8114,26183,5,0.800000,0.400000
8115,51001,13947,0.420449,0.298344


Hacemos el merge igual que en el caso anterior

In [34]:
votos_edades_segsol_paro = pd.merge(votos_edades_segsol, paro_simp, how="left", on=['cod_mun'])

In [35]:
votos_edades_segsol_paro

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,95-99,100 y más,Total,Hombres,Mujeres,Total_afiliados,prop_autonomos,total Dtes Empleo,prop_hombres,prop_mayores_45
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,1.0,0.0,1465.0,765.0,700.0,281.0,0.241993,186,0.553763,0.500000
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,0.0,1.0,1360.0,699.0,661.0,387.0,0.224806,227,0.427313,0.594714
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,1.0,0.0,966.0,459.0,507.0,7161.0,0.357213,3821,0.525255,0.369013
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,2.0,0.0,1579.0,789.0,790.0,7161.0,0.357213,3821,0.525255,0.369013
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,0.0,0.0,1789.0,871.0,918.0,7161.0,0.357213,3821,0.525255,0.369013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36058,022011111195200108010,19,52,52001,Melilla,Melilla,Melilla,1208,775,8,...,0.0,0.0,1480.0,735.0,745.0,19893.0,0.161816,14080,0.407173,0.319176
36059,022011111195200108011,19,52,52001,Melilla,Melilla,Melilla,1374,811,8,...,0.0,0.0,2107.0,1057.0,1050.0,19893.0,0.161816,14080,0.407173,0.319176
36060,022011111195200108012,19,52,52001,Melilla,Melilla,Melilla,1874,1183,13,...,1.0,0.0,2532.0,1307.0,1225.0,19893.0,0.161816,14080,0.407173,0.319176
36061,022011111195200108013,19,52,52001,Melilla,Melilla,Melilla,1167,637,7,...,1.0,0.0,1762.0,974.0,788.0,19893.0,0.161816,14080,0.407173,0.319176


Pasamos a la renta de las secciones, que cargamos sin novedad. Aquí los datos son por secciones.

In [36]:
strings_ren_sec = {'cod_sec' : 'str'}

In [37]:
import botocore.exceptions

KEY = 'INE/rentas_secciones_15_17.txt' 

try:
  
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'rentas_secciones_15_17.txt')

except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [38]:
ren_sec = pd.read_csv('rentas_secciones_15_17.txt', dtype = strings_ren_sec)

In [39]:
ren_sec

,Territorio,cod_sec,RNM per 2017,RNM per 2015,RNM hog 2017,RNM hog 2015
0,0100101001 Alegría-Dulantzi sección 01001,0100101001,11234.267197,10618.182737,28322.021999,26938.114416
1,0100101002 Alegría-Dulantzi sección 01002,0100101002,11234.267197,10618.182737,28322.021999,26938.114416
2,0100201001 Amurrio sección 01001,0100201001,13862.000000,13800.000000,34411.000000,34421.000000
3,0100201002 Amurrio sección 01002,0100201002,13862.000000,13800.000000,34411.000000,34421.000000
4,0100201003 Amurrio sección 01003,0100201003,13862.000000,13800.000000,34411.000000,34421.000000
...,...,...,...,...,...,...
36444,5200108011 Melilla sección 08011,5200108011,16433.000000,15847.000000,66352.000000,62632.000000
36445,5200108012 Melilla sección 08012,5200108012,17350.000000,16792.000000,50730.000000,50839.000000
36446,5200108013 Melilla sección 08013,5200108013,12553.000000,11823.000000,37816.000000,36729.000000
36447,5200108014 Melilla sección 08014,5200108014,8906.000000,8937.000000,29898.000000,31384.000000


In [40]:
ren_sec.columns

Index(['Territorio', 'cod_sec', 'RNM per 2017', 'RNM per 2015', 'RNM hog 2017',
       'RNM hog 2015'],
      dtype='object')

Nos quedamos con los datos de las rentas, y el código de sección...

In [41]:
ren_sec_simp = ren_sec[['cod_sec', 'RNM per 2017', 'RNM per 2015', 'RNM hog 2017',
       'RNM hog 2015']]

In [42]:
ren_sec_simp

,cod_sec,RNM per 2017,RNM per 2015,RNM hog 2017,RNM hog 2015
0,0100101001,11234.267197,10618.182737,28322.021999,26938.114416
1,0100101002,11234.267197,10618.182737,28322.021999,26938.114416
2,0100201001,13862.000000,13800.000000,34411.000000,34421.000000
3,0100201002,13862.000000,13800.000000,34411.000000,34421.000000
4,0100201003,13862.000000,13800.000000,34411.000000,34421.000000
...,...,...,...,...,...
36444,5200108011,16433.000000,15847.000000,66352.000000,62632.000000
36445,5200108012,17350.000000,16792.000000,50730.000000,50839.000000
36446,5200108013,12553.000000,11823.000000,37816.000000,36729.000000
36447,5200108014,8906.000000,8937.000000,29898.000000,31384.000000


... y hacemos simplemente otro merge.

In [43]:
votos_edades_segsol_paro_rensec = pd.merge(votos_edades_segsol_paro, ren_sec_simp, how="left", on=['cod_sec'])

In [44]:
votos_edades_segsol_paro_rensec

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,Mujeres,Total_afiliados,prop_autonomos,total Dtes Empleo,prop_hombres,prop_mayores_45,RNM per 2017,RNM per 2015,RNM hog 2017,RNM hog 2015
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,700.0,281.0,0.241993,186,0.553763,0.500000,9159.0,8788.0,20172.0,19546.0
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,661.0,387.0,0.224806,227,0.427313,0.594714,8827.0,8107.0,17841.0,17115.0
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,507.0,7161.0,0.357213,3821,0.525255,0.369013,8965.0,8267.0,26498.0,24688.0
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,790.0,7161.0,0.357213,3821,0.525255,0.369013,8599.0,7941.0,25677.0,23400.0
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,918.0,7161.0,0.357213,3821,0.525255,0.369013,8076.0,7150.0,22051.0,19687.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36058,022011111195200108010,19,52,52001,Melilla,Melilla,Melilla,1208,775,8,...,745.0,19893.0,0.161816,14080,0.407173,0.319176,14275.0,14155.0,45725.0,46108.0
36059,022011111195200108011,19,52,52001,Melilla,Melilla,Melilla,1374,811,8,...,1050.0,19893.0,0.161816,14080,0.407173,0.319176,16433.0,15847.0,66352.0,62632.0
36060,022011111195200108012,19,52,52001,Melilla,Melilla,Melilla,1874,1183,13,...,1225.0,19893.0,0.161816,14080,0.407173,0.319176,17350.0,16792.0,50730.0,50839.0
36061,022011111195200108013,19,52,52001,Melilla,Melilla,Melilla,1167,637,7,...,788.0,19893.0,0.161816,14080,0.407173,0.319176,12553.0,11823.0,37816.0,36729.0


Finalmente cargamos el dataset de la distribución de rentas por conceptos; también tiene los datos por secciones.

In [45]:
strings_dist_ren_sec = {'cod_sec' : 'str'}

In [46]:
import botocore.exceptions

KEY = 'INE/Dist_renta_hogares_15_18.txt' 

try:
  
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'Dist_renta_hogares_15_18.txt')

except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [47]:
dist_ren_sec = pd.read_csv('Dist_renta_hogares_15_18.txt', dtype = strings_dist_ren_sec)

In [48]:
dist_ren_sec

,Territorio,cod_sec,Sal 2018,Sal 2015,Pen 2018,Pen 2015,Desem 2018,Desem 2015
0,1190301001 San Martín del Tesorillo sección 01001,1190301001,4903.0,6845.948425,1959.0,2985.302533,321.0,347.217589
1,1190301002 San Martín del Tesorillo sección 01002,1190301002,4396.0,6845.948425,2093.0,2985.302533,318.0,347.217589
2,1490101001 Fuente Carreteros sección 01001,1490101001,3669.0,6845.948425,2267.0,2985.302533,846.0,347.217589
3,"1490201001 Guijarrosa, La sección 01001",1490201001,4288.0,6845.948425,2201.0,2985.302533,846.0,347.217589
4,1807701001 Fornes sección 01001,1807701001,4170.0,6845.948425,3049.0,2985.302533,813.0,347.217589
...,...,...,...,...,...,...,...,...
34283,5200108011 Melilla sección 08011,5200108011,11378.0,11119.000000,1508.0,1274.000000,167.0,166.000000
34284,5200108012 Melilla sección 08012,5200108012,13272.0,13038.000000,2763.0,2445.000000,169.0,177.000000
34285,5200108013 Melilla sección 08013,5200108013,10102.0,9640.000000,1807.0,1615.000000,234.0,252.000000
34286,5200108014 Melilla sección 08014,5200108014,5923.0,6061.000000,2463.0,2136.000000,244.0,284.000000


In [49]:
dist_ren_sec.columns

Index(['Territorio', 'cod_sec', 'Sal 2018', 'Sal 2015', 'Pen 2018', 'Pen 2015',
       'Desem 2018', 'Desem 2015'],
      dtype='object')

Tras quedarnos con las columnas realmente útiles, y el código de sección...

In [50]:
dist_ren_sec_simp = dist_ren_sec[[ 'cod_sec', 'Sal 2018', 'Sal 2015', 'Pen 2018', 'Pen 2015',
       'Desem 2018', 'Desem 2015']]

In [51]:
dist_ren_sec_simp

,cod_sec,Sal 2018,Sal 2015,Pen 2018,Pen 2015,Desem 2018,Desem 2015
0,1190301001,4903.0,6845.948425,1959.0,2985.302533,321.0,347.217589
1,1190301002,4396.0,6845.948425,2093.0,2985.302533,318.0,347.217589
2,1490101001,3669.0,6845.948425,2267.0,2985.302533,846.0,347.217589
3,1490201001,4288.0,6845.948425,2201.0,2985.302533,846.0,347.217589
4,1807701001,4170.0,6845.948425,3049.0,2985.302533,813.0,347.217589
...,...,...,...,...,...,...,...
34283,5200108011,11378.0,11119.000000,1508.0,1274.000000,167.0,166.000000
34284,5200108012,13272.0,13038.000000,2763.0,2445.000000,169.0,177.000000
34285,5200108013,10102.0,9640.000000,1807.0,1615.000000,234.0,252.000000
34286,5200108014,5923.0,6061.000000,2463.0,2136.000000,244.0,284.000000


...ejecutamos el merge.

In [52]:
votos_edades_segsol_paro_rensec_distrensec = pd.merge(votos_edades_segsol_paro_rensec, dist_ren_sec_simp, how="left", on=['cod_sec'])

In [53]:
votos_edades_segsol_paro_rensec_distrensec

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,RNM per 2017,RNM per 2015,RNM hog 2017,RNM hog 2015,Sal 2018,Sal 2015,Pen 2018,Pen 2015,Desem 2018,Desem 2015
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,9159.0,8788.0,20172.0,19546.0,5574.0,4833.0,3286.0,3082.0,403.0,471.0
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,8827.0,8107.0,17841.0,17115.0,4640.0,4048.0,3418.0,2770.0,568.0,620.0
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,8965.0,8267.0,26498.0,24688.0,5121.0,4795.0,2499.0,2301.0,337.0,333.0
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,8599.0,7941.0,25677.0,23400.0,5381.0,4837.0,1815.0,1724.0,343.0,464.0
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,8076.0,7150.0,22051.0,19687.0,5224.0,4044.0,1170.0,1198.0,416.0,476.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36058,022011111195200108010,19,52,52001,Melilla,Melilla,Melilla,1208,775,8,...,14275.0,14155.0,45725.0,46108.0,10429.0,11134.0,2861.0,2629.0,165.0,155.0
36059,022011111195200108011,19,52,52001,Melilla,Melilla,Melilla,1374,811,8,...,16433.0,15847.0,66352.0,62632.0,11378.0,11119.0,1508.0,1274.0,167.0,166.0
36060,022011111195200108012,19,52,52001,Melilla,Melilla,Melilla,1874,1183,13,...,17350.0,16792.0,50730.0,50839.0,13272.0,13038.0,2763.0,2445.0,169.0,177.0
36061,022011111195200108013,19,52,52001,Melilla,Melilla,Melilla,1167,637,7,...,12553.0,11823.0,37816.0,36729.0,10102.0,9640.0,1807.0,1615.0,234.0,252.0


Ahora ya hemos unificado las bases de datos, y hacemos una primera grabación.

In [54]:
votos_edades_segsol_paro_rensec_distrensec.to_csv('gen_N11_unif_cols_prov.txt', sep = ',', index = False)

Pero no hemos terminado, porque podemos comenzar a añadir columnas en función de las existentes, que nos resulten útiles. Una evidente es la participación electoral.

Pandas por defecto va colocando las nuevas columnas a la derecha del todo; posteriormente ya las colocaremos como nos venga mejor.

In [55]:
votos_edades_segsol_paro_rensec_distrensec['Participación'] = votos_edades_segsol_paro_rensec_distrensec['Votos_Total'] / votos_edades_segsol_paro_rensec_distrensec['Censo_Esc']  

In [56]:
votos_edades_segsol_paro_rensec_distrensec

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,RNM per 2015,RNM hog 2017,RNM hog 2015,Sal 2018,Sal 2015,Pen 2018,Pen 2015,Desem 2018,Desem 2015,Participación
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,8788.0,20172.0,19546.0,5574.0,4833.0,3286.0,3082.0,403.0,471.0,0.781387
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,8107.0,17841.0,17115.0,4640.0,4048.0,3418.0,2770.0,568.0,620.0,0.777487
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,8267.0,26498.0,24688.0,5121.0,4795.0,2499.0,2301.0,337.0,333.0,0.777299
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,7941.0,25677.0,23400.0,5381.0,4837.0,1815.0,1724.0,343.0,464.0,0.724763
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,7150.0,22051.0,19687.0,5224.0,4044.0,1170.0,1198.0,416.0,476.0,0.725780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36058,022011111195200108010,19,52,52001,Melilla,Melilla,Melilla,1208,775,8,...,14155.0,45725.0,46108.0,10429.0,11134.0,2861.0,2629.0,165.0,155.0,0.641556
36059,022011111195200108011,19,52,52001,Melilla,Melilla,Melilla,1374,811,8,...,15847.0,66352.0,62632.0,11378.0,11119.0,1508.0,1274.0,167.0,166.0,0.590247
36060,022011111195200108012,19,52,52001,Melilla,Melilla,Melilla,1874,1183,13,...,16792.0,50730.0,50839.0,13272.0,13038.0,2763.0,2445.0,169.0,177.0,0.631270
36061,022011111195200108013,19,52,52001,Melilla,Melilla,Melilla,1167,637,7,...,11823.0,37816.0,36729.0,10102.0,9640.0,1807.0,1615.0,234.0,252.0,0.545844


In [57]:
votos_edades_segsol_paro_rensec_distrensec.columns

Index(['Sección', 'cod_ccaa', 'cod_prov', 'cod_mun', 'CCAA', 'Provincia',
       'Municipio', 'Censo_Esc', 'Votos_Total', 'Nulos', 'Votos_Válidos',
       'Blanco', 'V_Cand', 'PP', 'PSOE', 'Cs', 'UP', 'IU', 'VOX', 'UPyD', 'MP',
       'CiU', 'ERC', 'JxC', 'CUP', 'DiL', 'PNV', 'Bildu', 'Amaiur', 'CC', 'FA',
       'TE', 'BNG', 'PRC', 'GBai', 'Compromis', 'PACMA', 'Otros', 'cod_sec',
       '0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
       '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79',
       '80-84', '85-89', '90-94', '95-99', '100 y más', 'Total', 'Hombres',
       'Mujeres', 'Total_afiliados', 'prop_autonomos', 'total Dtes Empleo',
       'prop_hombres', 'prop_mayores_45', 'RNM per 2017', 'RNM per 2015',
       'RNM hog 2017', 'RNM hog 2015', 'Sal 2018', 'Sal 2015', 'Pen 2018',
       'Pen 2015', 'Desem 2018', 'Desem 2015', 'Participación'],
      dtype='object')

Ahora vamos a definir una serie de columnas que nos digan qué partido ganó en cada sección, y quién quedó segundo, tercero, cuarto y quinto. 

Ello lleva varios pasos, comenzando con la creación de un diccionario que almacene los resultados electorales en formato partido : votos.

In [58]:
partidos = ['PP', 'PSOE', 'Cs', 'UP', 'IU', 'VOX', 'UPyD', 'MP',
       'CiU', 'ERC', 'JxC', 'CUP', 'DiL', 'PNV', 'Bildu', 'Amaiur', 'CC', 'FA',
       'TE', 'BNG', 'PRC', 'GBai', 'Compromis', 'PACMA', 'Otros']

In [59]:
votos_edades_segsol_paro_rensec_distrensec['dict_res'] = ''

In [60]:
for ind in range(len(votos_edades_segsol_paro_rensec_distrensec['Sección'])):
  votos_edades_segsol_paro_rensec_distrensec['dict_res'][ind] = {p : votos_edades_segsol_paro_rensec_distrensec[p][ind] for p in partidos } 


<ipython-input-60-a916be5943e5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votos_edades_segsol_paro_rensec_distrensec['dict_res'][ind] = {p : votos_edades_segsol_paro_rensec_distrensec[p][ind] for p in partidos }


In [61]:
votos_edades_segsol_paro_rensec_distrensec

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,RNM hog 2017,RNM hog 2015,Sal 2018,Sal 2015,Pen 2018,Pen 2015,Desem 2018,Desem 2015,Participación,dict_res
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,20172.0,19546.0,5574.0,4833.0,3286.0,3082.0,403.0,471.0,0.781387,"{'PP': 434, 'PSOE': 373, 'Cs': 0, 'UP': 0, 'IU..."
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,17841.0,17115.0,4640.0,4048.0,3418.0,2770.0,568.0,620.0,0.777487,"{'PP': 361, 'PSOE': 454, 'Cs': 0, 'UP': 0, 'IU..."
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,26498.0,24688.0,5121.0,4795.0,2499.0,2301.0,337.0,333.0,0.777299,"{'PP': 364, 'PSOE': 129, 'Cs': 0, 'UP': 0, 'IU..."
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,25677.0,23400.0,5381.0,4837.0,1815.0,1724.0,343.0,464.0,0.724763,"{'PP': 547, 'PSOE': 223, 'Cs': 0, 'UP': 0, 'IU..."
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,22051.0,19687.0,5224.0,4044.0,1170.0,1198.0,416.0,476.0,0.725780,"{'PP': 598, 'PSOE': 195, 'Cs': 0, 'UP': 0, 'IU..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36058,022011111195200108010,19,52,52001,Melilla,Melilla,Melilla,1208,775,8,...,45725.0,46108.0,10429.0,11134.0,2861.0,2629.0,165.0,155.0,0.641556,"{'PP': 541, 'PSOE': 146, 'Cs': 0, 'UP': 0, 'IU..."
36059,022011111195200108011,19,52,52001,Melilla,Melilla,Melilla,1374,811,8,...,66352.0,62632.0,11378.0,11119.0,1508.0,1274.0,167.0,166.0,0.590247,"{'PP': 535, 'PSOE': 187, 'Cs': 0, 'UP': 0, 'IU..."
36060,022011111195200108012,19,52,52001,Melilla,Melilla,Melilla,1874,1183,13,...,50730.0,50839.0,13272.0,13038.0,2763.0,2445.0,169.0,177.0,0.631270,"{'PP': 843, 'PSOE': 245, 'Cs': 0, 'UP': 0, 'IU..."
36061,022011111195200108013,19,52,52001,Melilla,Melilla,Melilla,1167,637,7,...,37816.0,36729.0,10102.0,9640.0,1807.0,1615.0,234.0,252.0,0.545844,"{'PP': 415, 'PSOE': 167, 'Cs': 0, 'UP': 0, 'IU..."


Vemos un ejemplo tomado al azar.

In [62]:
a = votos_edades_segsol_paro_rensec_distrensec['dict_res'][852]

In [63]:
a

{'PP': 140,
 'PSOE': 174,
 'Cs': 0,
 'UP': 0,
 'IU': 42,
 'VOX': 0,
 'UPyD': 20,
 'MP': 0,
 'CiU': 0,
 'ERC': 0,
 'JxC': 0,
 'CUP': 0,
 'DiL': 0,
 'PNV': 0,
 'Bildu': 0,
 'Amaiur': 0,
 'CC': 0,
 'FA': 0,
 'TE': 0,
 'BNG': 0,
 'PRC': 0,
 'GBai': 0,
 'Compromis': 0,
 'PACMA': 3,
 'Otros': 25}

Ahora lo tenemos que ordenar por los votos de manera decreciente, almecenando los datos en un array de tuplas. Este sería un ejemplo en base al diccionario anterior

In [64]:
b = sorted(a.items(), key=lambda item: item[1], reverse = True)

In [65]:
b

[('PSOE', 174),
 ('PP', 140),
 ('IU', 42),
 ('Otros', 25),
 ('UPyD', 20),
 ('PACMA', 3),
 ('Cs', 0),
 ('UP', 0),
 ('VOX', 0),
 ('MP', 0),
 ('CiU', 0),
 ('ERC', 0),
 ('JxC', 0),
 ('CUP', 0),
 ('DiL', 0),
 ('PNV', 0),
 ('Bildu', 0),
 ('Amaiur', 0),
 ('CC', 0),
 ('FA', 0),
 ('TE', 0),
 ('BNG', 0),
 ('PRC', 0),
 ('GBai', 0),
 ('Compromis', 0)]

In [66]:
b[0]

('PSOE', 174)

In [67]:
b[0][0]

'PSOE'

Ahora iteramos por los diccionarios de cada registro, y los depositamos en una columna, en este caso la de 'dict_res_ord'.

In [68]:
votos_edades_segsol_paro_rensec_distrensec['dict_res_ord'] = ''

In [69]:
for ind in range(len(votos_edades_segsol_paro_rensec_distrensec['Sección'])):

  a = votos_edades_segsol_paro_rensec_distrensec['dict_res'][ind]
  votos_edades_segsol_paro_rensec_distrensec['dict_res_ord'][ind] = sorted(a.items(), key=lambda item: item[1], reverse = True)



<ipython-input-69-c3b6b9e6a48d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votos_edades_segsol_paro_rensec_distrensec['dict_res_ord'][ind] = sorted(a.items(), key=lambda item: item[1], reverse = True)


Vemos el ejemplo tomado ya del propio dataset. Parece correcto.

In [70]:
votos_edades_segsol_paro_rensec_distrensec['dict_res_ord'][852]

[('PSOE', 174),
 ('PP', 140),
 ('IU', 42),
 ('Otros', 25),
 ('UPyD', 20),
 ('PACMA', 3),
 ('Cs', 0),
 ('UP', 0),
 ('VOX', 0),
 ('MP', 0),
 ('CiU', 0),
 ('ERC', 0),
 ('JxC', 0),
 ('CUP', 0),
 ('DiL', 0),
 ('PNV', 0),
 ('Bildu', 0),
 ('Amaiur', 0),
 ('CC', 0),
 ('FA', 0),
 ('TE', 0),
 ('BNG', 0),
 ('PRC', 0),
 ('GBai', 0),
 ('Compromis', 0)]

Identificamos al ganador con el doble índice [0][0].

Análogamente, veremos que el segundo partido tendrá un doble índice [1][0], y así sucesivamente.

In [71]:
votos_edades_segsol_paro_rensec_distrensec['dict_res_ord'][852][0][0]

'PSOE'

Tenemos que crear las columnas que almacenarán los puestos de cada sección.

In [72]:
votos_edades_segsol_paro_rensec_distrensec['Ganador'] = ''
votos_edades_segsol_paro_rensec_distrensec['Segundo'] = ''
votos_edades_segsol_paro_rensec_distrensec['Tercero'] = ''
votos_edades_segsol_paro_rensec_distrensec['Cuarto'] = ''
votos_edades_segsol_paro_rensec_distrensec['Quinto'] = ''


Ahora metemos en la columna Ganador al partido ganador de la sección. Lógicamente, en noviembre de 2011 el PP ganó en la gran mayoría de ellas.

In [73]:
votos_edades_segsol_paro_rensec_distrensec['Ganador'] = votos_edades_segsol_paro_rensec_distrensec['dict_res_ord'].apply(lambda x : x[0][0])

In [74]:
votos_edades_segsol_paro_rensec_distrensec['Ganador'].value_counts()

PP        25459
PSOE       6731
CiU        2668
PNV         594
Amaiur      532
IU           27
CC           17
Otros        15
GBai          7
ERC           5
BNG           5
UPyD          1
PACMA         1
PRC           1
Name: Ganador, dtype: int64

Ahora vamos con el que quedó segundo, en este caso el PSOE.

In [75]:
votos_edades_segsol_paro_rensec_distrensec['Segundo'] = votos_edades_segsol_paro_rensec_distrensec['dict_res_ord'].apply(lambda x : x[1][0])

In [76]:
votos_edades_segsol_paro_rensec_distrensec['Segundo'].value_counts()

PSOE      26312
PP         6797
PNV         635
CiU         631
Amaiur      453
ERC         285
CC          262
IU          181
Otros       160
UPyD        140
GBai        113
BNG          51
PRC          40
PACMA         3
Name: Segundo, dtype: int64

Y finalmente rellenamos los puestos tercero, cuarto y quinto.

In [77]:
votos_edades_segsol_paro_rensec_distrensec['Tercero'] = votos_edades_segsol_paro_rensec_distrensec['dict_res_ord'].apply(lambda x : x[2][0])

In [78]:
votos_edades_segsol_paro_rensec_distrensec['Cuarto'] = votos_edades_segsol_paro_rensec_distrensec['dict_res_ord'].apply(lambda x : x[3][0])

In [79]:
votos_edades_segsol_paro_rensec_distrensec['Quinto'] = votos_edades_segsol_paro_rensec_distrensec['dict_res_ord'].apply(lambda x : x[4][0])

In [80]:
votos_edades_segsol_paro_rensec_distrensec['Quinto'].value_counts()

Otros     19774
UPyD       6584
IU         5544
ERC        1440
PACMA       987
Cs          973
UP          369
PP          138
PSOE         58
CC           54
Amaiur       47
CiU          47
GBai         40
PRC           6
PNV           2
Name: Quinto, dtype: int64

Las columnas de los puestos pandas las coloca a la derecha del todo por defecto.

In [81]:
votos_edades_segsol_paro_rensec_distrensec

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,Desem 2018,Desem 2015,Participación,dict_res,dict_res_ord,Ganador,Segundo,Tercero,Cuarto,Quinto
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,403.0,471.0,0.781387,"{'PP': 434, 'PSOE': 373, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 434), (PSOE, 373), (IU, 34), (UPyD, 27),...",PP,PSOE,IU,UPyD,Otros
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,568.0,620.0,0.777487,"{'PP': 361, 'PSOE': 454, 'Cs': 0, 'UP': 0, 'IU...","[(PSOE, 454), (PP, 361), (IU, 40), (UPyD, 19),...",PSOE,PP,IU,UPyD,Otros
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,337.0,333.0,0.777299,"{'PP': 364, 'PSOE': 129, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 364), (PSOE, 129), (UPyD, 14), (IU, 13),...",PP,PSOE,UPyD,IU,Otros
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,343.0,464.0,0.724763,"{'PP': 547, 'PSOE': 223, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 547), (PSOE, 223), (UPyD, 23), (Otros, 1...",PP,PSOE,UPyD,Otros,IU
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,416.0,476.0,0.725780,"{'PP': 598, 'PSOE': 195, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 598), (PSOE, 195), (IU, 35), (UPyD, 28),...",PP,PSOE,IU,UPyD,Otros
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36058,022011111195200108010,19,52,52001,Melilla,Melilla,Melilla,1208,775,8,...,165.0,155.0,0.641556,"{'PP': 541, 'PSOE': 146, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 541), (PSOE, 146), (UPyD, 43), (Otros, 1...",PP,PSOE,UPyD,Otros,PACMA
36059,022011111195200108011,19,52,52001,Melilla,Melilla,Melilla,1374,811,8,...,167.0,166.0,0.590247,"{'PP': 535, 'PSOE': 187, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 535), (PSOE, 187), (UPyD, 44), (Otros, 1...",PP,PSOE,UPyD,Otros,PACMA
36060,022011111195200108012,19,52,52001,Melilla,Melilla,Melilla,1874,1183,13,...,169.0,177.0,0.631270,"{'PP': 843, 'PSOE': 245, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 843), (PSOE, 245), (UPyD, 37), (Otros, 3...",PP,PSOE,UPyD,Otros,PACMA
36061,022011111195200108013,19,52,52001,Melilla,Melilla,Melilla,1167,637,7,...,234.0,252.0,0.545844,"{'PP': 415, 'PSOE': 167, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 415), (PSOE, 167), (UPyD, 21), (Otros, 1...",PP,PSOE,UPyD,Otros,PACMA


Estas son las columnas que llevamos, 86.

In [82]:
a = list(votos_edades_segsol_paro_rensec_distrensec.columns)
a

['Sección',
 'cod_ccaa',
 'cod_prov',
 'cod_mun',
 'CCAA',
 'Provincia',
 'Municipio',
 'Censo_Esc',
 'Votos_Total',
 'Nulos',
 'Votos_Válidos',
 'Blanco',
 'V_Cand',
 'PP',
 'PSOE',
 'Cs',
 'UP',
 'IU',
 'VOX',
 'UPyD',
 'MP',
 'CiU',
 'ERC',
 'JxC',
 'CUP',
 'DiL',
 'PNV',
 'Bildu',
 'Amaiur',
 'CC',
 'FA',
 'TE',
 'BNG',
 'PRC',
 'GBai',
 'Compromis',
 'PACMA',
 'Otros',
 'cod_sec',
 '0-4',
 '5-9',
 '10-14',
 '15-19',
 '20-24',
 '25-29',
 '30-34',
 '35-39',
 '40-44',
 '45-49',
 '50-54',
 '55-59',
 '60-64',
 '65-69',
 '70-74',
 '75-79',
 '80-84',
 '85-89',
 '90-94',
 '95-99',
 '100 y más',
 'Total',
 'Hombres',
 'Mujeres',
 'Total_afiliados',
 'prop_autonomos',
 'total Dtes Empleo',
 'prop_hombres',
 'prop_mayores_45',
 'RNM per 2017',
 'RNM per 2015',
 'RNM hog 2017',
 'RNM hog 2015',
 'Sal 2018',
 'Sal 2015',
 'Pen 2018',
 'Pen 2015',
 'Desem 2018',
 'Desem 2015',
 'Participación',
 'dict_res',
 'dict_res_ord',
 'Ganador',
 'Segundo',
 'Tercero',
 'Cuarto',
 'Quinto']

Otra columna interesante puede ser la suma de los mayores de edad, que podemos definir como los mayores de 64 años, y así la definimos y la calculamos.

In [83]:
votos_edades_segsol_paro_rensec_distrensec['mayores_65_años'] = 0

In [84]:
mayores = ['65-69', '70-74', '75-79','80-84', '85-89', '90-94', '95-99', '100 y más']

In [85]:
votos_edades_segsol_paro_rensec_distrensec['mayores_65_años'] = votos_edades_segsol_paro_rensec_distrensec[mayores].sum(axis=1)

In [86]:
votos_edades_segsol_paro_rensec_distrensec['mayores_65_años']

0        359.0
1        336.0
2        143.0
3        153.0
4        200.0
         ...  
36058    203.0
36059    158.0
36060    319.0
36061    152.0
36062    155.0
Name: mayores_65_años, Length: 36063, dtype: float64

Podemos hacer lo mismo con los menores de edad; es cierto que éstos no votan, pero puede ser un factor importante en los que só lo hacen.

In [87]:
votos_edades_segsol_paro_rensec_distrensec['menores_19_años'] = 0

In [88]:
menores = ['0-4','5-9', '10-14','15-19']

In [89]:
votos_edades_segsol_paro_rensec_distrensec['menores_19_años'] = votos_edades_segsol_paro_rensec_distrensec[menores].sum(axis=1)

In [90]:
votos_edades_segsol_paro_rensec_distrensec['menores_19_años'] 

0        228.0
1        192.0
2        206.0
3        374.0
4        437.0
         ...  
36058    298.0
36059    620.0
36060    531.0
36061    353.0
36062    294.0
Name: menores_19_años, Length: 36063, dtype: float64

Por otro lado, podemos definir una columna con una especie de tasa de paro, que podemos definir como los demandantes de empleo entre la suma de los demandantes de empleo y los afiliados a la Seguridad Social.

En general, debemos dar una cierta prioridad a los datos que sean proporciones y datos absolutos, pues los primeros se pueden comparar independientemente del tamaño de la población.

In [91]:
votos_edades_segsol_paro_rensec_distrensec['Desempleo'] = votos_edades_segsol_paro_rensec_distrensec['total Dtes Empleo'] / (votos_edades_segsol_paro_rensec_distrensec['total Dtes Empleo'] + votos_edades_segsol_paro_rensec_distrensec['Total_afiliados'] )

In [92]:
votos_edades_segsol_paro_rensec_distrensec

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,dict_res,dict_res_ord,Ganador,Segundo,Tercero,Cuarto,Quinto,mayores_65_años,menores_19_años,Desempleo
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,"{'PP': 434, 'PSOE': 373, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 434), (PSOE, 373), (IU, 34), (UPyD, 27),...",PP,PSOE,IU,UPyD,Otros,359.0,228.0,0.398287
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,"{'PP': 361, 'PSOE': 454, 'Cs': 0, 'UP': 0, 'IU...","[(PSOE, 454), (PP, 361), (IU, 40), (UPyD, 19),...",PSOE,PP,IU,UPyD,Otros,336.0,192.0,0.369707
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,"{'PP': 364, 'PSOE': 129, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 364), (PSOE, 129), (UPyD, 14), (IU, 13),...",PP,PSOE,UPyD,IU,Otros,143.0,206.0,0.347933
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,"{'PP': 547, 'PSOE': 223, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 547), (PSOE, 223), (UPyD, 23), (Otros, 1...",PP,PSOE,UPyD,Otros,IU,153.0,374.0,0.347933
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,"{'PP': 598, 'PSOE': 195, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 598), (PSOE, 195), (IU, 35), (UPyD, 28),...",PP,PSOE,IU,UPyD,Otros,200.0,437.0,0.347933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36058,022011111195200108010,19,52,52001,Melilla,Melilla,Melilla,1208,775,8,...,"{'PP': 541, 'PSOE': 146, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 541), (PSOE, 146), (UPyD, 43), (Otros, 1...",PP,PSOE,UPyD,Otros,PACMA,203.0,298.0,0.414447
36059,022011111195200108011,19,52,52001,Melilla,Melilla,Melilla,1374,811,8,...,"{'PP': 535, 'PSOE': 187, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 535), (PSOE, 187), (UPyD, 44), (Otros, 1...",PP,PSOE,UPyD,Otros,PACMA,158.0,620.0,0.414447
36060,022011111195200108012,19,52,52001,Melilla,Melilla,Melilla,1874,1183,13,...,"{'PP': 843, 'PSOE': 245, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 843), (PSOE, 245), (UPyD, 37), (Otros, 3...",PP,PSOE,UPyD,Otros,PACMA,319.0,531.0,0.414447
36061,022011111195200108013,19,52,52001,Melilla,Melilla,Melilla,1167,637,7,...,"{'PP': 415, 'PSOE': 167, 'Cs': 0, 'UP': 0, 'IU...","[(PP, 415), (PSOE, 167), (UPyD, 21), (Otros, 1...",PP,PSOE,UPyD,Otros,PACMA,152.0,353.0,0.414447


In [93]:
votos_edades_segsol_paro_rensec_distrensec['Desempleo'].describe()

count    36002.000000
mean         0.276865
std          0.119931
min          0.000000
25%          0.190700
50%          0.265596
75%          0.352664
max          0.970017
Name: Desempleo, dtype: float64

En este momento hacemos una especie parada y fonda, guardando el dataset una vez más, y volviéndolo a cargar.

In [95]:
votos_edades_segsol_paro_rensec_distrensec.to_csv('gen_N11_unif_cols_prov.txt', sep = ',', index = False)

In [96]:
cols_str = {'cod_ccaa' : 'str', 'cod_prov' : 'str', 'cod_mun' : 'str', 'cod_sec' : 'str', 'cod_grupo' : 'str'}

In [97]:
gen_xx_cols = pd.read_csv('gen_N11_unif_cols_prov.txt', dtype = cols_str)

In [98]:
gen_xx_cols.head()

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,dict_res,dict_res_ord,Ganador,Segundo,Tercero,Cuarto,Quinto,mayores_65_años,menores_19_años,Desempleo
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,"{'PP': 434, 'PSOE': 373, 'Cs': 0, 'UP': 0, 'IU...","[('PP', 434), ('PSOE', 373), ('IU', 34), ('UPy...",PP,PSOE,IU,UPyD,Otros,359.0,228.0,0.398287
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,"{'PP': 361, 'PSOE': 454, 'Cs': 0, 'UP': 0, 'IU...","[('PSOE', 454), ('PP', 361), ('IU', 40), ('UPy...",PSOE,PP,IU,UPyD,Otros,336.0,192.0,0.369707
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,"{'PP': 364, 'PSOE': 129, 'Cs': 0, 'UP': 0, 'IU...","[('PP', 364), ('PSOE', 129), ('UPyD', 14), ('I...",PP,PSOE,UPyD,IU,Otros,143.0,206.0,0.347933
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,"{'PP': 547, 'PSOE': 223, 'Cs': 0, 'UP': 0, 'IU...","[('PP', 547), ('PSOE', 223), ('UPyD', 23), ('O...",PP,PSOE,UPyD,Otros,IU,153.0,374.0,0.347933
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,"{'PP': 598, 'PSOE': 195, 'Cs': 0, 'UP': 0, 'IU...","[('PP', 598), ('PSOE', 195), ('IU', 35), ('UPy...",PP,PSOE,IU,UPyD,Otros,200.0,437.0,0.347933


Podemos seguir optimizando las columnas, cambiando los datos absolutos por proporciones, como hemos mencionado anteriormente, en este caso con los mayores y menores de edad.

In [99]:
gen_xx_cols['mayores_65_años'] = gen_xx_cols['mayores_65_años'] / gen_xx_cols['Total']

In [100]:
gen_xx_cols['menores_19_años'] = gen_xx_cols['menores_19_años'] / gen_xx_cols['Total']

In [101]:
gen_xx_cols.head()

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,dict_res,dict_res_ord,Ganador,Segundo,Tercero,Cuarto,Quinto,mayores_65_años,menores_19_años,Desempleo
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,"{'PP': 434, 'PSOE': 373, 'Cs': 0, 'UP': 0, 'IU...","[('PP', 434), ('PSOE', 373), ('IU', 34), ('UPy...",PP,PSOE,IU,UPyD,Otros,0.245051,0.155631,0.398287
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,"{'PP': 361, 'PSOE': 454, 'Cs': 0, 'UP': 0, 'IU...","[('PSOE', 454), ('PP', 361), ('IU', 40), ('UPy...",PSOE,PP,IU,UPyD,Otros,0.247059,0.141176,0.369707
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,"{'PP': 364, 'PSOE': 129, 'Cs': 0, 'UP': 0, 'IU...","[('PP', 364), ('PSOE', 129), ('UPyD', 14), ('I...",PP,PSOE,UPyD,IU,Otros,0.148033,0.213251,0.347933
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,"{'PP': 547, 'PSOE': 223, 'Cs': 0, 'UP': 0, 'IU...","[('PP', 547), ('PSOE', 223), ('UPyD', 23), ('O...",PP,PSOE,UPyD,Otros,IU,0.096897,0.236859,0.347933
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,"{'PP': 598, 'PSOE': 195, 'Cs': 0, 'UP': 0, 'IU...","[('PP', 598), ('PSOE', 195), ('IU', 35), ('UPy...",PP,PSOE,IU,UPyD,Otros,0.111794,0.244271,0.347933


Ahora debemos definir el porcentaje de voto de cada partido como se hace en realidad: votos a un partido entre el total de votos válidos. Almecenamos esta proporción en columnas paracada partido, tomando los más importantes.

In [102]:
gen_xx_cols['% PP'] = gen_xx_cols['PP'] / gen_xx_cols['Votos_Válidos']

In [103]:
gen_xx_cols['% PSOE'] = gen_xx_cols['PSOE'] / gen_xx_cols['Votos_Válidos']

In [104]:
gen_xx_cols['% UP'] = gen_xx_cols['UP'] / gen_xx_cols['Votos_Válidos']

In [105]:
gen_xx_cols['% Vox'] = gen_xx_cols['VOX'] / gen_xx_cols['Votos_Válidos']

In [106]:
gen_xx_cols['% Cs'] = gen_xx_cols['Cs'] / gen_xx_cols['Votos_Válidos']

In [107]:
gen_xx_cols['% IU'] = gen_xx_cols['IU'] / gen_xx_cols['Votos_Válidos']

In [108]:
gen_xx_cols.head()

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,Quinto,mayores_65_años,menores_19_años,Desempleo,% PP,% PSOE,% UP,% Vox,% Cs,% IU
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,Otros,0.245051,0.155631,0.398287,0.490950,0.421946,0.0,0.0,0.0,0.038462
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,Otros,0.247059,0.141176,0.369707,0.408371,0.513575,0.0,0.0,0.0,0.045249
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,Otros,0.148033,0.213251,0.347933,0.681648,0.241573,0.0,0.0,0.0,0.024345
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,IU,0.096897,0.236859,0.347933,0.656663,0.267707,0.0,0.0,0.0,0.019208
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,Otros,0.111794,0.244271,0.347933,0.685780,0.223624,0.0,0.0,0.0,0.040138


Ahora comenzamos a cambiar de nombres algunas columnas para hacerlas más reconocibles e interpretables.

In [109]:
gen_xx_cols.rename(columns = {'Total_afiliados' : 'Afiliados SS Minicipio'})

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,Quinto,mayores_65_años,menores_19_años,Desempleo,% PP,% PSOE,% UP,% Vox,% Cs,% IU
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,Otros,0.245051,0.155631,0.398287,0.490950,0.421946,0.0,0.0,0.0,0.038462
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,Otros,0.247059,0.141176,0.369707,0.408371,0.513575,0.0,0.0,0.0,0.045249
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,Otros,0.148033,0.213251,0.347933,0.681648,0.241573,0.0,0.0,0.0,0.024345
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,IU,0.096897,0.236859,0.347933,0.656663,0.267707,0.0,0.0,0.0,0.019208
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,Otros,0.111794,0.244271,0.347933,0.685780,0.223624,0.0,0.0,0.0,0.040138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36058,022011111195200108010,19,52,52001,Melilla,Melilla,Melilla,1208,775,8,...,PACMA,0.137162,0.201351,0.414447,0.705346,0.190352,0.0,0.0,0.0,0.000000
36059,022011111195200108011,19,52,52001,Melilla,Melilla,Melilla,1374,811,8,...,PACMA,0.074988,0.294257,0.414447,0.666252,0.232877,0.0,0.0,0.0,0.000000
36060,022011111195200108012,19,52,52001,Melilla,Melilla,Melilla,1874,1183,13,...,PACMA,0.125987,0.209716,0.414447,0.720513,0.209402,0.0,0.0,0.0,0.000000
36061,022011111195200108013,19,52,52001,Melilla,Melilla,Melilla,1167,637,7,...,PACMA,0.086266,0.200341,0.414447,0.658730,0.265079,0.0,0.0,0.0,0.000000


Seguimos cambiando nombres de columnas...

In [110]:
gen_xx_cols = gen_xx_cols.rename(columns = {'Total_afiliados' : 'Afiliados SS Minicipio', 'prop_autonomos' : '% Afiliados SS autónomos', 'total Dtes Empleo	' : 'Paro registrado Municipio', 'prop_hombres	' : '% Paro Hombres'})

In [111]:
gen_xx_cols = gen_xx_cols.rename(columns = {'prop_mayores_45' : '% Paro mayores 45', 'RNM per 2017' : 'Renta persona 2017', 'RNM per 2015' : 'Renta persona 2015', 'RNM hog 2017' : 'Renta hogar 2017' , 'RNM hog 2015' : 'Renta hogar 2015'})

In [112]:
gen_xx_cols = gen_xx_cols.rename(columns = {'Sal 2018' : 'Renta Salarios 2018', 'Sal 2015' : 'Renta Salarios 2015', 'Pen 2018' : 'Renta Pensiones 2018', 'Pen 2015' : 'Renta Pensiones 2015' , 'Desem 2018' : 'Renta Desempleo 2018', 'Desem 2015' : 'Renta Desempleo 2015'})

In [113]:
gen_xx_cols.head()

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,Quinto,mayores_65_años,menores_19_años,Desempleo,% PP,% PSOE,% UP,% Vox,% Cs,% IU
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,Otros,0.245051,0.155631,0.398287,0.490950,0.421946,0.0,0.0,0.0,0.038462
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,Otros,0.247059,0.141176,0.369707,0.408371,0.513575,0.0,0.0,0.0,0.045249
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,Otros,0.148033,0.213251,0.347933,0.681648,0.241573,0.0,0.0,0.0,0.024345
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,IU,0.096897,0.236859,0.347933,0.656663,0.267707,0.0,0.0,0.0,0.019208
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,Otros,0.111794,0.244271,0.347933,0.685780,0.223624,0.0,0.0,0.0,0.040138


... y seguimos.

In [114]:
gen_xx_cols = gen_xx_cols.rename(columns = {'total Dtes Empleo' : 'Paro Registrado Municipio', 'prop_hombres' : '% Paro Hombres', 'Desempleo' : '% Paro s/ Afiliados SS Municipio'})

In [115]:
gen_xx_cols.head()

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,Quinto,mayores_65_años,menores_19_años,% Paro s/ Afiliados SS Municipio,% PP,% PSOE,% UP,% Vox,% Cs,% IU
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,Otros,0.245051,0.155631,0.398287,0.490950,0.421946,0.0,0.0,0.0,0.038462
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,Otros,0.247059,0.141176,0.369707,0.408371,0.513575,0.0,0.0,0.0,0.045249
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,Otros,0.148033,0.213251,0.347933,0.681648,0.241573,0.0,0.0,0.0,0.024345
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,IU,0.096897,0.236859,0.347933,0.656663,0.267707,0.0,0.0,0.0,0.019208
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,Otros,0.111794,0.244271,0.347933,0.685780,0.223624,0.0,0.0,0.0,0.040138


In [116]:
gen_xx_cols = gen_xx_cols.rename(columns = {'mayores_65_años' : '% mayores_65_años', 'menores_19_años' : '% menores_19_años'})

Añadimos una columna de edades intermedias que es resta de las de menores y mayores de 65 años.

In [117]:
gen_xx_cols['% 20-64 años'] = 1 - gen_xx_cols['% mayores_65_años'] - gen_xx_cols['% menores_19_años']

In [118]:
gen_xx_cols.head()

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,% mayores_65_años,% menores_19_años,% Paro s/ Afiliados SS Municipio,% PP,% PSOE,% UP,% Vox,% Cs,% IU,% 20-64 años
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,0.245051,0.155631,0.398287,0.490950,0.421946,0.0,0.0,0.0,0.038462,0.599317
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,0.247059,0.141176,0.369707,0.408371,0.513575,0.0,0.0,0.0,0.045249,0.611765
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,0.148033,0.213251,0.347933,0.681648,0.241573,0.0,0.0,0.0,0.024345,0.638716
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,0.096897,0.236859,0.347933,0.656663,0.267707,0.0,0.0,0.0,0.019208,0.666244
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,0.111794,0.244271,0.347933,0.685780,0.223624,0.0,0.0,0.0,0.040138,0.643935


Añadimos otra columna, la de la proporcion de afiliados a la SS del total del municipio, que creemos que podría ser una variable importante y estar correlacionada con el voto.

In [119]:
gen_xx_cols['% Afiliados SS / Población'] = gen_xx_cols['Afiliados SS Minicipio'] / gen_xx_cols['Total'] 

In [120]:
gen_xx_cols = gen_xx_cols.rename(columns = {'Total' : 'Población Total', '% menores_19_años' : '% menores 19 años', '% mayores_65_años' : '% mayores 65 años', })

In [121]:
gen_xx_cols.head()

,Sección,cod_ccaa,cod_prov,cod_mun,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Nulos,...,% menores 19 años,% Paro s/ Afiliados SS Municipio,% PP,% PSOE,% UP,% Vox,% Cs,% IU,% 20-64 años,% Afiliados SS / Población
0,022011111010400101001,01,04,04001,Andalucía,Almería,Abla,1139,890,6,...,0.155631,0.398287,0.490950,0.421946,0.0,0.0,0.0,0.038462,0.599317,0.191809
1,022011111010400201001,01,04,04002,Andalucía,Almería,Abrucena,1146,891,7,...,0.141176,0.369707,0.408371,0.513575,0.0,0.0,0.0,0.045249,0.611765,0.284559
2,022011111010400301001,01,04,04003,Andalucía,Almería,Adra,696,541,7,...,0.213251,0.347933,0.681648,0.241573,0.0,0.0,0.0,0.024345,0.638716,7.413043
3,022011111010400301002,01,04,04003,Andalucía,Almería,Adra,1159,840,7,...,0.236859,0.347933,0.656663,0.267707,0.0,0.0,0.0,0.019208,0.666244,4.535149
4,022011111010400301003,01,04,04003,Andalucía,Almería,Adra,1218,884,12,...,0.244271,0.347933,0.685780,0.223624,0.0,0.0,0.0,0.040138,0.643935,4.002795


Ya no tenemos más columnas que añadir, por lo que nos ponemos a ordenarlas de forma conveniente, con las definiciones geográficas a la izquierda, seguida de los datos electorales, y finalmente los sociodemográficos. 

In [122]:
gen_xx_cols.columns

Index(['Sección', 'cod_ccaa', 'cod_prov', 'cod_mun', 'CCAA', 'Provincia',
       'Municipio', 'Censo_Esc', 'Votos_Total', 'Nulos', 'Votos_Válidos',
       'Blanco', 'V_Cand', 'PP', 'PSOE', 'Cs', 'UP', 'IU', 'VOX', 'UPyD', 'MP',
       'CiU', 'ERC', 'JxC', 'CUP', 'DiL', 'PNV', 'Bildu', 'Amaiur', 'CC', 'FA',
       'TE', 'BNG', 'PRC', 'GBai', 'Compromis', 'PACMA', 'Otros', 'cod_sec',
       '0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
       '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79',
       '80-84', '85-89', '90-94', '95-99', '100 y más', 'Población Total',
       'Hombres', 'Mujeres', 'Afiliados SS Minicipio',
       '% Afiliados SS autónomos', 'Paro Registrado Municipio',
       '% Paro Hombres', '% Paro mayores 45', 'Renta persona 2017',
       'Renta persona 2015', 'Renta hogar 2017', 'Renta hogar 2015',
       'Renta Salarios 2018', 'Renta Salarios 2015', 'Renta Pensiones 2018',
       'Renta Pensiones 2015', 'Renta Desempleo 2018'

In [123]:
ord_cols = ['Sección', 'cod_ccaa', 'cod_prov', 'cod_mun', 'cod_sec', 'CCAA', 'Provincia',
       'Municipio', 'Censo_Esc', 'Votos_Total','Participación', 'Nulos', 'Votos_Válidos',
       'Blanco', 'V_Cand', 'PP', 'PSOE', 'Cs', 'UP', 'IU', 'VOX', 'UPyD', 'MP',
       'CiU', 'ERC', 'JxC', 'CUP', 'DiL', 'PNV', 'Bildu', 'Amaiur', 'CC', 'FA',
       'TE', 'BNG', 'PRC', 'GBai', 'Compromis', 'PACMA', 'Otros', '% PP',
       '% PSOE', '% UP', '% Vox', '% Cs', '% IU', 'Ganador',
       'Segundo', 'Tercero', 'Cuarto', 'Quinto',
       '0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
       '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79',
       '80-84', '85-89', '90-94', '95-99', '100 y más', 'Población Total',
       'Hombres', 'Mujeres',  '% mayores 65 años', '% 20-64 años',
       '% menores 19 años','Afiliados SS Minicipio',
       '% Afiliados SS autónomos', '% Afiliados SS / Población', 'Paro Registrado Municipio',
       '% Paro Hombres', '% Paro mayores 45', '% Paro s/ Afiliados SS Municipio',  'Renta persona 2017',
       'Renta persona 2015', 'Renta hogar 2017', 'Renta hogar 2015',
       'Renta Salarios 2018', 'Renta Salarios 2015', 'Renta Pensiones 2018',
       'Renta Pensiones 2015', 'Renta Desempleo 2018', 'Renta Desempleo 2015',
       'dict_res', 'dict_res_ord']

In [124]:
len(ord_cols)

97

In [125]:
gen_xx_cols = gen_xx_cols[ord_cols]

In [126]:
gen_xx_cols.head()

,Sección,cod_ccaa,cod_prov,cod_mun,cod_sec,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,...,Renta hogar 2017,Renta hogar 2015,Renta Salarios 2018,Renta Salarios 2015,Renta Pensiones 2018,Renta Pensiones 2015,Renta Desempleo 2018,Renta Desempleo 2015,dict_res,dict_res_ord
0,022011111010400101001,01,04,04001,0400101001,Andalucía,Almería,Abla,1139,890,...,20172.0,19546.0,5574.0,4833.0,3286.0,3082.0,403.0,471.0,"{'PP': 434, 'PSOE': 373, 'Cs': 0, 'UP': 0, 'IU...","[('PP', 434), ('PSOE', 373), ('IU', 34), ('UPy..."
1,022011111010400201001,01,04,04002,0400201001,Andalucía,Almería,Abrucena,1146,891,...,17841.0,17115.0,4640.0,4048.0,3418.0,2770.0,568.0,620.0,"{'PP': 361, 'PSOE': 454, 'Cs': 0, 'UP': 0, 'IU...","[('PSOE', 454), ('PP', 361), ('IU', 40), ('UPy..."
2,022011111010400301001,01,04,04003,0400301001,Andalucía,Almería,Adra,696,541,...,26498.0,24688.0,5121.0,4795.0,2499.0,2301.0,337.0,333.0,"{'PP': 364, 'PSOE': 129, 'Cs': 0, 'UP': 0, 'IU...","[('PP', 364), ('PSOE', 129), ('UPyD', 14), ('I..."
3,022011111010400301002,01,04,04003,0400301002,Andalucía,Almería,Adra,1159,840,...,25677.0,23400.0,5381.0,4837.0,1815.0,1724.0,343.0,464.0,"{'PP': 547, 'PSOE': 223, 'Cs': 0, 'UP': 0, 'IU...","[('PP', 547), ('PSOE', 223), ('UPyD', 23), ('O..."
4,022011111010400301003,01,04,04003,0400301003,Andalucía,Almería,Adra,1218,884,...,22051.0,19687.0,5224.0,4044.0,1170.0,1198.0,416.0,476.0,"{'PP': 598, 'PSOE': 195, 'Cs': 0, 'UP': 0, 'IU...","[('PP', 598), ('PSOE', 195), ('IU', 35), ('UPy..."


Finalmente guardamos el dataset, que será el que utilizamos para modelar. Nos dimos cuenta al casi acabar la práctica que la columna de las CCAA están mal, ya que el INE y el Ministerio del Interior utilizan distincos códigos para ellas. Intentaremos corregirlo, a fin de cuentas se trataría de hacer un mapeo.

Obviamente, hay un dataset por cada elección.

In [127]:
gen_xx_cols.to_csv('gen_N11_unif_cols_prov.txt', sep = ',', index = False)

In [128]:
#para guardar el archivo en s3:

from botocore.exceptions import ClientError

s3_client = boto3.client(
    's3',
    aws_access_key_id='XXXXXX',
    aws_secret_access_key='XXXXXXX',    
)

def upload_file(file_name, bucket, object_name=None):
    """Subir un archivo a un bucket
    :param file_name: archivo que hay que subir
    :param bucket: Bucket al que hay que subirlo
    :param object_name: S3 object name. Incluye la carpeta en la que hay que guardarlo. si no hay no se pone nada
    :return: True si sube el archivo, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    #s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [132]:
upload_file('gen_N11_unif_cols_prov.txt',
            'electomedia',
            object_name = "datos-elecciones-generales-unificados/" + 'gen_N11_unif_cols_prov_copia.txt'
           )

True